<a href="https://colab.research.google.com/github/iggy2k/CSC490-Project-Winter-2025/blob/main/Cords_then_geocell_490stencil_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# !tar -xf /content/drive/My\ Drive/conda_colab.tar.gz -C ../

# import os
# path = '/content/miniconda3/bin:' + os.environ['PATH']
# %env PATH=$path
# %env PYTHONPATH=
# import sys
# _ = sys.path.append("/content/miniconda3/lib/python3.7/site-packages")

In [2]:
!python --version

Python 3.11.11


In [3]:
# This is supposed to make sure if you edit .py files
# colab will pick up on it
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys

# Local
if os.path.isdir('PIGEON/'):
    PIGEON_DIR = 'PIGEON/'
# Git clone when colab
else:
    PIGEON_DIR = 'PROJECT/PIGEON/'

os.environ["PIGEON_DIR"] = PIGEON_DIR
sys.path.insert(1, PIGEON_DIR)

In [5]:
!if [[ ! -d PROJECT ]] && [[ ! -d PIGEON ]]; then git clone https://github.com/iggy2k/CSC490-Project-Winter-2025.git PROJECT; fi

Cloning into 'PROJECT'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 386 (delta 15), reused 25 (delta 10), pack-reused 352 (from 1)
Receiving objects: 100% (386/386), 24.75 MiB | 29.30 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [6]:
import os
import yaml
from tqdm.notebook import tqdm
import subprocess



with open("./PROJECT/environment.yaml") as file_handle:
    environment_data = yaml.safe_load(file_handle)

for dependency in tqdm(environment_data["dependencies"], total=len(environment_data["dependencies"])):
    if isinstance(dependency, dict):
      for lib in tqdm(dependency['pip'], total=len(dependency['pip'])):
        try:
            out = subprocess.check_output(f'pip install {lib.split("=")[0]}', shell=True)
        except subprocess.CalledProcessError as err:
            print(err)
    else:
      try:
          subprocess.check_output(f'pip install {dependency.split("=")[0]}', shell=True)
      except subprocess.CalledProcessError as err:
          print(err)
try:
  subprocess.check_output('pip install pycountry global_land_mask', shell=True)
except subprocess.CalledProcessError as err:
  print(err)

  0%|          | 0/26 [00:00<?, ?it/s]

Command 'pip install matplotlib-base' returned non-zero exit status 1.
Command 'pip install python' returned non-zero exit status 1.
Command 'pip install pytorch' returned non-zero exit status 1.


  0%|          | 0/18 [00:00<?, ?it/s]

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')
# !tar -zcf conda_colab.tar.gz /content/miniconda3
# !cp conda_colab.tar.gz /content/drive/My\ Drive/

[autoreload of jupyter_client.manager failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: _laun

In [8]:
from huggingface_hub import snapshot_download
import os

if not os.path.isdir('datasets/osv5m/images'):
    snapshot_download(repo_id="osv5m/osv5m", local_dir="datasets/osv5m", allow_patterns=[
        'images/train/00.zip',
        'images/test/00.zip',
        '*.csv'
        ], repo_type='dataset')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

00.zip:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/116M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

00.zip:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

In [9]:
import csv
import numpy as np
import pandas as pd
from shapely.geometry import Point

import geopandas as gpd
from geopandas import GeoDataFrame

import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import zipfile
from time import sleep

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torchvision import datasets, transforms

from PIL import Image
import pandas as pd

from tqdm.notebook import trange, tqdm

In [10]:
for root, dirs, files in os.walk("datasets/osv5m"):
    for file in files:
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(root, file), 'r') as zip_ref:
                for member in tqdm(zip_ref.infolist(), desc=f'Extracting {os.path.join(root, file)}'):
                  zip_ref.extract(member, root)
            os.remove(os.path.join(root, file))

Extracting datasets/osv5m/images/test/00.zip:   0%|          | 0/50001 [00:00<?, ?it/s]

Extracting datasets/osv5m/images/train/00.zip:   0%|          | 0/50001 [00:00<?, ?it/s]

In [11]:
import pycountry

# Debug
MAX_ITEMS = 1000

class ImageCoordinateDataset(Dataset):
    def __init__(self, csv_file, image_dirs, transform=None):
        self.data = []
        self.files = []
        self.images = []
        self.skipped = 0
        self.csv = csv_file
        self.transform = transform

        for image_dir in image_dirs:
          print('Reading', image_dir)

          self.files.extend([f"{image_dir}/{f}" for f in listdir(image_dir) if isfile(join(image_dir, f))])

          print(f'Found {len(self.files)} files.')

          if not os.path.isdir('datasets/osv5m/'):
            os.makedirs('datasets/osv5m/')
          if isfile(f"{csv_file}_filtered.csv"):
            self.df = pd.read_csv(f"{csv_file}_filtered.csv", index_col=False)
          else:
            self.df = pd.concat([chunk for chunk in tqdm(pd.read_csv(self.csv, chunksize=5000, usecols=['id', 'latitude', 'longitude', 'country'], index_col=False), desc='Loading data')])

          print(f'Found {len(self.df)} csv entries.')

          self.df['country'] = self.df['country'].apply(lambda x: pycountry.countries.get(alpha_2=x).name if pycountry.countries.get(alpha_2=x) else x)

          new = pd.DataFrame(columns=['id', 'latitude', 'longitude', 'country'])
          i = 0
          for full_path in tqdm(self.files, total=len(self.files), desc='Processing files'):
              image_name = str(Path(full_path).stem)

              try:
                row = self.df[self.df['id'] == int(image_name)].iloc[0]
              except:
                continue
              new.loc[i] = row
              lat = row['latitude']
              lon = row['longitude']

              # Remove mislaballed images (ocean pictures?)
              # if not globe.is_land(float(lat), float(lon)):
              #   self.skipped += 1
              #   continue
              self.data.append(np.array([str(full_path), float(lat), float(lon)]))
              i += 1

          self.df = new
        if MAX_ITEMS is not None:
          print(f'Keeping {MAX_ITEMS} items')
          self.df = self.df[:MAX_ITEMS]
          self.data = self.data[:MAX_ITEMS]
          self.files = self.files[:MAX_ITEMS]

        self.df.to_csv(f"{csv_file}_filtered.csv", index=False)

        print(f'Dataset ready, {len(self.files)} files.')
        print(f'Skipped {self.skipped} non-land files.')


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx][0]
        coordinates = (float(self.data[idx][1]), float(self.data[idx][2]))
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(coordinates, dtype=torch.float32)


In [12]:
# https://www.geeksforgeeks.org/how-to-normalize-images-in-pytorch/
# The mean and std of ImageNet are: mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
normalize = transforms.Normalize(mean.tolist(), std.tolist())

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

In [13]:
BATCH_SIZE = 32

In [14]:
train_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/train.csv', image_dirs=[
                                      'datasets/osv5m/images/train/00',
                                      ],\
                                 transform=transform)
val_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/test.csv', image_dirs=[
                                      'datasets/osv5m/images/test/00',
                                      ],\
                                 transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

Reading datasets/osv5m/images/train/00
Found 50000 files.


Loading data: 0it [00:00, ?it/s]

Found 4894684 csv entries.


Processing files:   0%|          | 0/50000 [00:00<?, ?it/s]

Keeping 1000 items
Dataset ready, 1000 files.
Skipped 0 non-land files.
Reading datasets/osv5m/images/test/00
Found 50000 files.


Loading data: 0it [00:00, ?it/s]

Found 210122 csv entries.


Processing files:   0%|          | 0/50000 [00:00<?, ?it/s]

Keeping 1000 items
Dataset ready, 1000 files.
Skipped 0 non-land files.


In [15]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('using cuda')
else:
  device = torch.device("cpu")
  print('using cpu')

using cuda


In [16]:
CLIP_MODEL = 'openai/clip-vit-base-patch32'

In [17]:
from transformers import AutoModelForImageClassification, CLIPVisionModel
embed_model = CLIPVisionModel.from_pretrained(CLIP_MODEL)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [18]:
!mkdir -p data/geocells/
# Political boundaries (admin0 = country)
![ ! -f data/geocells/admin_2.geojson ] && wget --show-progress -O data/geocells/admin_2.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.geojson
![ ! -f data/geocells/admin_1.geojson ] && wget --show-progress -O data/geocells/admin_1.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.geojson
![ ! -f data/geocells/countries.geojson ] && wget --show-progress -O data/geocells/countries.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.geojson

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

data/geocells/admin 100%[===================>] 524.95M   354MB/s    in 1.5s    
data/geocells/admin 100%[===================>] 343.82M   342MB/s    in 1.0s    
data/geocells/count 100%[===================>] 382.61M   333MB/s    in 1.1s    


In [19]:
# TODO: pre-generate all this, add to the repo and add option to load instead
import sys
sys.path.append(f'{PIGEON_DIR}/dataset_creation/geocell')
sys.path.append(PIGEON_DIR)

from geocell_creation import *

GEOCELL_PATH = 'data/geocells_yfcc.csv'
df = train_dataset.df
geocells_file = Path(GEOCELL_PATH)
if not geocells_file.is_file():
    geocell_creator = GeocellCreator(df, GEOCELL_PATH)
    geocells = geocell_creator.generate()




/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Loading geographic boundaries ...
 ... loaded admin 2 boundaries.
 ... loaded admin 1 boundaries.
 ... loaded countries.


Fusing cells within countries: 100%|██████████| 100/100 [07:55<00:00,  4.75s/country]


||| NEW OPTICS PARAMS |||  (300, 0.05)
||| NEW OPTICS PARAMS |||  (400, 0.005)
||| NEW OPTICS PARAMS |||  (1000, 0.0001)


In [20]:
# !pip install -q -U srtm-py georasters pandarallel latlon-utils pygeos datasets huggingface_hub

In [21]:
!pip install -q huggingface-hub transformers pygeos

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.7 MB/s eta 0:00:00


In [22]:
!pip install -U dill datasets

In [32]:
import torchvision
from torchvision import datasets, transforms

import torch
import pandas as pd
from torch import nn, Tensor
from torch.nn.parameter import Parameter
from collections import namedtuple
from preprocessing import haversine_matrix, smooth_labels
from models.layers import PositionalEncoder
from models.utils import ModelOutput
from config import *
GEOCELL_PATH = 'data/geocells_yfcc.csv'

#TODO: move over more stuff from PIGEON

class GeoLocationModel(nn.Module):
    def __init__(self):
        super(GeoLocationModel, self).__init__()

        self.base_model = embed_model  # CLIP-based vision model

        resnet = torchvision.models.resnet50(pretrained=True).to(device)
        for i, param in enumerate(resnet.parameters()):
            if i < 4:
                param.requires_grad = False

        self._set_hidden_size()

        geocell_path = GEOCELL_PATH
        self.lla_geocells = self.load_geocells(geocell_path)
        self.num_cells = self.lla_geocells.size(0)

        self.input_dim = self.hidden_size

        # **New layer for predicting coordinates (lon, lat)**
        self.coord_layer = nn.Linear(self.input_dim, 2)  # Outputs (longitude, latitude)

        # Geocell classification layer (used for refinement)
        self.cell_layer = nn.Linear(self.input_dim, self.num_cells)
        self.softmax = nn.Softmax(dim=-1)

        # Add missing num_candidates attribute**
        self.num_candidates = 5  # Define number of top geocell predictions

        # Loss for coordinate regression
        self.loss_fnc = nn.MSELoss()  # Use Mean Squared Error for coordinate loss



    def forward(self, x):
      embedding = self.base_model(pixel_values=x)

      if embedding.pooler_output is None:
        print("⚠️ Embedding output is None! Replacing with zeros.")
        embedding.pooler_output = torch.zeros((x.shape[0], self.hidden_size)).to(x.device)  # Fix NoneType issue


      # 🔍 Debug: Check if embedding has NaNs
      if torch.isnan(embedding.pooler_output).any():
          print("⚠️ NaN detected in embedding! Returning None.")
          return None, None  # Prevents NaNs from propagating

      # Select embedding mode
      if self.mode == 'transformer':
          embedding = embedding.last_hidden_state
          embedding = torch.mean(embedding, dim=1)
      else:
          embedding = embedding.pooler_output

      output = embedding  # Store for later

      # Predict coordinates (lon, lat)
      pred_LLH = self.coord_layer(output)

      # ✅ Debug: Check for NaNs in coordinate output
      if torch.isnan(pred_LLH).any() or torch.isinf(pred_LLH).any():
          print("⚠️ NaN detected in coordinate output! Setting to zero.")
          pred_LLH = torch.nan_to_num(pred_LLH, nan=0.0)  # Replace NaNs with 0

      # Predict geocell logits and apply clamping
      logits = self.cell_layer(output).clamp(min=-100, max=100)  # ✅ Keeps values stable
      geocell_probs = self.softmax(logits)

      # ✅ Debug: Check for NaNs in geocell probabilities
      if torch.isnan(geocell_probs).any() or torch.isinf(geocell_probs).any():
          print("⚠️ NaN detected in geocell probabilities! Setting to zero.")
          geocell_probs = torch.nan_to_num(geocell_probs, nan=0.0)  # Replace NaNs with 0

      # Get top-k geocell predictions
      geocell_topk = torch.topk(geocell_probs, self.num_candidates, dim=-1)

      return pred_LLH, geocell_topk  # ✅ Now returns geocell again




    def load_geocells(self, path: str) -> Tensor:
        """Loads geocell centroids and converts them to ECEF format

        Args:
            path (str, optional): path to geocells. Defaults to GEOCELL_PATH.

        Returns:
            Tensor: ECEF geocell centroids
        """
        geo_df = pd.read_csv(path)
        lla_coords = torch.tensor(geo_df[['longitude', 'latitude']].values)
        lla_geocells = nn.parameter.Parameter(data=lla_coords, requires_grad=False)
        return lla_geocells

    def _set_hidden_size(self):
        """
        Determines the hidden size of the model
        """
        if self.base_model is not None:
            try:
                self.hidden_size = self.base_model.config.hidden_size
                self.mode = 'transformer'

            except AttributeError:
                self.hidden_size = self.base_model.config.hidden_sizes[-1]
                self.mode = 'convnext'

    # def _freeze_params(self):
    #     """Freezes model parameters depending on mode
    #     """
    #     if self.base_model is not None:
    #         if self.freeze_base:
    #             for param in self.base_model.parameters():
    #                 param.requires_grad = False

    #         # Load parameters and freeze relevant parameters
    #         elif 'clip-vit' in self.base_model.config._name_or_path and not self.serving:
    #             head = CLIP_PRETRAINED_HEAD_YFCC if self.yfcc else CLIP_PRETRAINED_HEAD
    #             self.load_state(head)
    #             print(f'Initialized model parameters from model: {head}')
    #             for param in self.base_model.vision_model.encoder.layers[:-1].parameters():
    #                 param.requires_grad = False

model = GeoLocationModel()

model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [33]:
from transformers import CLIPProcessor, CLIPModel

In [34]:
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
clip_processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
clip_model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, 

In [35]:
# https://github.com/gastruc/osv5m/blob/main/models/losses.py
class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, x, y):
        """
        Args:
            x: torch.Tensor Bx2
            y: torch.Tensor Bx2
        Returns:
            torch.Tensor: Haversine loss between x and y: torch.Tensor([B])
        Note:
            Haversine distance doesn't contain the 2 * 6371 constant.
        """
        lhs = torch.sin((x[:, 0] - y[:, 0]) / 2) ** 2
        rhs = (
            torch.cos(x[:, 0])
            * torch.cos(y[:, 0])
            * torch.sin((x[:, 1] - y[:, 1]) / 2) ** 2
        )
        a = lhs + rhs
        return torch.arctan2(torch.sqrt(a), torch.sqrt(1 - a))

In [36]:
from torch.optim.lr_scheduler import ExponentialLR

haversineLoss = HaversineLoss()
mseLoss = nn.MSELoss()
ceLoss = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9, verbose=True)

In [37]:
!pip install -q livelossplot

In [38]:
from livelossplot import PlotLosses

In [39]:
from ipywidgets import Output
OUTPUT_CONTEXT = Output()
display(OUTPUT_CONTEXT)

Output()

In [40]:
train_loss = []  # Track losses
avg_epoch_loss = []
num_epochs = 5

for epoch in tqdm(range(num_epochs), total=num_epochs, desc='Epochs'):
    train_loss = []

    for images, coordinates in tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1} progress'):
        images, coordinates = images.to(device), coordinates.to(device)

        optimizer.zero_grad()

        pred_LLH, geocell_topk = model(images)  # Get coordinate & geocell predictions

        if pred_LLH is None or geocell_topk is None:  # ✅ Skip batch safely
          print("⚠️ Model returned None, skipping batch.")
          continue

        if torch.isnan(pred_LLH).any() or torch.isinf(pred_LLH).any():  # ✅ Now this is safe
          print(f"⚠️ Warning: Model output contains NaN or inf! Batch Skipped.")
          continue

        print(f"pred_LLH shape: {pred_LLH.shape}, coordinates shape: {coordinates.shape}")


        # Compute coordinate loss (MSELoss for lat/lon regression)
        loss_coord = mseLoss(pred_LLH, coordinates)

        print(f"pred_LLH shape: {pred_LLH.shape}, coordinates shape: {coordinates.shape}")

        train_loss.append(loss_coord.item())  # ✅ FIX: Track loss
        loss_coord.backward()

        for param in model.parameters():
            if torch.isnan(param).any() or torch.isinf(param).any():
                print("⚠️ Warning: Model weights contain NaN or Inf! Stopping training.")
                raise ValueError("NaN detected in model parameters")

        optimizer.step()

    scheduler.step()
    avg_loss = sum(train_loss) / len(train_loss)  # ✅ FIX: Now train_loss is not empty
    avg_epoch_loss.append(avg_loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')


Epoch 1 progress:   3%|▎         | 1/32 [00:00<00:11,  2.69it/s]

⚠️ NaN detected in embedding! Returning None.
⚠️ Model returned None, skipping batch.



Epoch 1 progress:   6%|▋         | 2/32 [00:00<00:10,  2.97it/s]

⚠️ NaN detected in embedding! Returning None.
⚠️ Model returned None, skipping batch.



Epoch 1 progress:   9%|▉         | 3/32 [00:00<00:09,  3.10it/s]

...


Epoch 1 progress:  94%|█████████▍| 30/32 [00:09<00:00,  3.38it/s]

⚠️ NaN detected in embedding! Returning None.
⚠️ Model returned None, skipping batch.



Epochs:   0%|          | 0/5 [00:10<?, ?it/s]

⚠️ NaN detected in embedding! Returning None.
⚠️ Model returned None, skipping batch.
⚠️ NaN detected in embedding! Returning None.
⚠️ Model returned None, skipping batch.


ZeroDivisionError: division by zero

In [ ]:
import matplotlib.pyplot as plt
avg_epoch_loss = torch.tensor(avg_epoch_loss, device =  'cpu')
plt.figure(figsize=(7, 5))
plt.plot(range(1, len(avg_epoch_loss) + 1), avg_epoch_loss, label='Loss', color='blue', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

In [ ]:
batch_haversine = []  # Initialize validation loss tracking
avg_val_haversine_loss = 0  # Reset validation loss

for images, true_coordinates in tqdm(val_dataloader, desc="Evaluating"):
    images, true_coordinates = images.to(device), true_coordinates.to(device)

    # Get predictions
    pred_LLH, geocell_topk = model(images)

    # Refine: Pick closest geocell centroid from top-k predictions
    best_geocell_index = geocell_topk.indices[:, 0]  # Select top geocell
    refined_LLH = model.lla_geocells[best_geocell_index]  # Use geocell centroid

    # Final prediction: Blend predicted lat/lon with geocell refinement
    final_pred_LLH = 0.8 * pred_LLH + 0.2 * refined_LLH

    # Compute Haversine loss for validation
    loss_haversine = haversineLoss(final_pred_LLH, true_coordinates).mean(dim=-1)

    # Track loss
    batch_haversine.append(loss_haversine.item())
    avg_val_haversine_loss += loss_haversine.item()

# Compute final average loss
avg_val_haversine_loss /= len(val_dataloader)
print(f"Average Validation Haversine Loss: {avg_val_haversine_loss:.5f}")


In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter

# Back from gpu
batch_haversine = torch.tensor(batch_haversine, device =  'cpu')

# Remove outliers
batch_haversine = median_filter(batch_haversine, size=100)

plt.figure(figsize=(5, 5))
plt.subplot(1, 1, 1)
plt.plot(range(1, len(batch_haversine) + 1), batch_haversine, label='Validation haversine', color='green', marker='o')
plt.xlabel('Batch Number')
plt.ylabel('haversine')
plt.title('Validation haversine loss per Batch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
from global_land_mask import globe
import matplotlib.pyplot as plt
import random

url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

unnormalize = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())

def plot_predictions(model, dataloader, num_samples=5):
    model.eval()
    with torch.no_grad():
        for images, coordinates in dataloader:

            images = images.cuda()

            pred_LLH, _ = model(images)

            images = images.cpu()
            pred_LLH = pred_LLH.cpu()

            rand_index = random.sample(range(0, len(images) - 1), min(num_samples, len(images) - 1))

            for i in range(min(num_samples, len(images))):
                i = rand_index[i]

                pred_lat, pred_lon = outputs[i].cpu().numpy()
                true_lat, true_lon = coordinates[i].numpy()

                haver_err = haversineLoss(
                                      torch.tensor(np.array([[pred_lon, pred_lat]]), dtype=torch.float32).deg2rad(),
                                      torch.tensor(np.array([[true_lon, true_lat]]), dtype=torch.float32).deg2rad(),
                                      )
                # Display the image
                img = images[i]
                img = unnormalize(img).permute(1, 2, 0).numpy()
                img = np.clip(img, 0, 1)

                plt.imshow(img)
                plt.title(f'Pred: ({pred_lat:.4f}, {pred_lon:.4f})\nTrue: ({true_lat:.4f}, {true_lon:.4f})\n Haversine: {haver_err}')
                plt.axis('off')

                # World map for better understanding of how bad our prediction is
                geometry = [Point(pred_lon, pred_lat), Point(true_lon, true_lat)]
                geo_df = GeoDataFrame(geometry = geometry)
                world = gpd.read_file(url)
                geo_df.plot(ax=world.plot(color="lightgrey", figsize=(10, 6)), marker='x', c=['red', 'green'], markersize=50);

                plt.show()
            break

In [ ]:
plot_predictions(model, val_dataloader, num_samples=10)